# TESTE PRÁTICO DE LINGUAGEM DE CONSULTA ESTRUTURADA (SQL)
Para a realização das atividades, você deverá executar as instruções abaixo para importar as bibliotecas e funções necessárias para criar a base de dados.

Ao final, esperamos que você desenvolva os códigos selecionados e entregue o notebook com as consultas SQL desenvolvidas e o arquivo .db criado.

O teste importará três arquivos cruciais para o desenvolvimento dessa atividade. Todos estão no Githun (não mude as URL's).

*   usuarios.csv
*   produtos.csv
*   vendas.csv

## O teste foi desenvolvido para ser executado com o SQLite3 e no Google Colab (Ubuntu 18.04). Você deverá utilizar consultas no padrão ANSI. Os comandos necessários podem ser encontrados em:

https://www.sqlite.org/docs.html

https://www.tutorialspoint.com/sqlite/index.htm



# AVISO!



1.   Execute o código abaixo para atualizar a biblioteca do SQLite3 utilizada para essa atividade
2.   **Ignore as mensagens de erro sobre perda de conectividade na instrução abaixo (o kernel será reiniciado para atualização do SQLite3)**


In [ ]:
import os

# BAIXAR A ÚLTIMA RELEASE DO COLAB, DESCOMPACTAR E INSTALAR NO DIRETÓRIO DO 
# PYTHON 3.7 (VERSÃO USADA PELO COLAB NESSE NOTEBOOK)
!curl https://www.sqlite.org/src/tarball/sqlite.tar.gz?r=release | tar xz
%cd sqlite/
!./configure
!make sqlite3.c
%cd /content/sqlite/
!npx degit coleifer/pysqlite3 -f
!cp sqlite/sqlite3.[ch] .
!python setup.py build_static build
!cp build/lib.linux-x86_64-3.7/pysqlite3/_sqlite3.cpython-37m-x86_64-linux-gnu.so \
     /usr/lib/python3.7/lib-dynload/

# REINICIANDO O KERNEL DO COLAB PARA IMPORTAR A BIBLIOTECA ATUALIZADA
os.kill(os.getpid(), 9)

# INSTRUÇÕES PARA REALIZAR AS CONSULTAS
### PARA CONSULTAR DADOS, SIGA OS PASSOS ABAIXO:

#### Abrir conexão:
    conexao_sql = sqlite3.connect(banco)
#### Abrir cursor para executar consulta no banco:
    cursor = conexao_sql.cursor()
#### Criar uma consulta:
    consulta = '''
        Coloque aqui a sua instrução SQL;
    '''
    cursor.execute(consulta)
#### Para visualizar o resultado da sua consulta:
    for linha in cursor:
      print(linha)
#### Para fechar a conexão:
    conexao_sql.close()

### LEMBRE-SE DE QUE PYTHON É SENSÍVEL A IDENTAÇÃO DE CÓDIGO!

# IMPORTANDO BIBLIOTECAS NECESSÁRIAS PARA EXECUTAR OS CÓDIGOS EM SQL

In [ ]:
import sqlite3
import csv
import requests
from pathlib import Path
from urllib.parse import urlparse

# DEFINIÇÃO DE VARIÁVEIS FIXAS PARA O PROJETO

In [ ]:
banco = "teste_sql.db"

# FUNÇÃO PARA INSERIR REGISTROS NO BANCO DE DADOS

In [ ]:
# Recebe como parâmetro
# 1. Nome do banco de dados
# 2. Nome da tabela que receberá os dados
# 3. Lista das colunas da tabela
# 4. Dados que serão inseridos

def inserir_registros(banco, tabela, lista_colunas, arquivo):
  conexao_sql = sqlite3.connect(banco)
  cursor = conexao_sql.cursor()
  dados = open(arquivo)
  conteudo_dados = csv.reader(dados)
  num_colunas = 1 if lista_colunas.count(',') == 1 else lista_colunas.count(',')
  string_insert = 'INSERT INTO ' + tabela + ' (' + lista_colunas + ') VALUES (?' + ('' if num_colunas == 1 else ', ?' * (num_colunas)) + ')'
  cursor.executemany(string_insert, conteudo_dados)
  conexao_sql.commit()
  conexao_sql.close()

# FUNÇÃO PARA BAIXAR A BASE DE DADOS UTILIZADA PARA DESENVOLVIMENTO DAS ATIVIDADES

In [ ]:
# Recebe como parâmetro:
# 1. Nome do arquivo com a extensão (Ex: base.csv)
# 2. URL do site onde o dado está localizado (Ex: 'https://drive.google.com/user_teste/folder_dataset/file.csv'):

def baixar_dados(nome_arquivo, url):
  requisicao = requests.get(url)
  conteudo = requisicao.content
  arquivo_csv = open(nome_arquivo, 'wb')
  arquivo_csv.write(conteudo)
  arquivo_csv.close()

# CRIAÇÃO DE CONEXÃO COM O BANCO SQLITE3 PARA DESENVOLVIMENTO DA ATIVIDADE

In [ ]:
# Abre conexão com o banco. Se o arquivo não existir, cria a base.
conexao_sql = sqlite3.connect(banco)
conexao_sql.close()

# CRIAÇÃO DAS TABELAS PARA REALIZAÇÃO DAS ATIVIDADES

### TABELA DE USUÁRIOS

In [ ]:
url = 'https://raw.githubusercontent.com/A3Data/testes_praticos/main/sql/base_dados/usuarios.csv'
nome_arquivo = Path(urlparse(url).path).name
baixar_dados(nome_arquivo, url)

conexao_sql = sqlite3.connect(banco)
# CRIAR A TABELA
conexao_sql.execute('''
CREATE TABLE IF NOT EXISTS usuarios
(
  cod_usuario INTEGER NOT NULL,
  data_cadastro DATE NOT NULL,
  faixa_etaria TEXT,
  cidade TEXT NOT NULL,
  estado TEXT NOT NULL);'''
)
conexao_sql.commit()

# INSERIR OS REGISTROS NO BANCO DE DADOS
lista_campos = 'cod_usuario, data_cadastro, faixa_etaria, cidade, estado'
inserir_registros(banco, 'usuarios', lista_campos, nome_arquivo)

### TABELA DE PRODUTOS

In [ ]:
url = 'https://raw.githubusercontent.com/A3Data/testes_praticos/main/sql/base_dados/produtos.csv'
nome_arquivo = Path(urlparse(url).path).name
baixar_dados(nome_arquivo, url)

conexao_sql = sqlite3.connect(banco)
arquivo_produtos = 'produtos.csv'
# CRIAR A TABELA
conexao_sql.execute('''
CREATE TABLE IF NOT EXISTS produtos
(
  cod_produto INTEGER NOT NULL,
  nome_produto TEXT NOT NULL,
  categoria_produto TEXT,
  valor_produto FLOAT);'''
)
conexao_sql.commit()

# INSERIR OS REGISTROS NO BANCO DE DADOS
lista_campos = 'cod_produto, nome_produto, categoria_produto, valor_produto'
inserir_registros(banco, 'produtos', lista_campos, arquivo_produtos)

### TABELA DE VENDAS

In [ ]:
url = 'https://raw.githubusercontent.com/A3Data/testes_praticos/main/sql/base_dados/vendas.csv'
nome_arquivo = Path(urlparse(url).path).name
baixar_dados(nome_arquivo, url)

conexao_sql = sqlite3.connect(banco)
arquivo_vendas = 'vendas.csv'
# CRIAR A TABELA
conexao_sql.execute('''
CREATE TABLE IF NOT EXISTS vendas
(
  cod_usuario INTEGER NOT NULL,
  cod_produto INTEGER NOT NULL,
  data_compra DATE NOT NULL,
  quantidade INT NOT NULL,
  valor FLOAT NOT NULL);'''
)
conexao_sql.commit()

# INSERIR OS REGISTROS NO BANCO DE DADOS
lista_campos = 'cod_usuario, cod_produto, data_compra, quantidade, valor'
inserir_registros(banco, 'vendas', lista_campos, arquivo_vendas)

# PERGUNTA 1:

### Escreva um comando em SQL que retorne a quantidade de vendas dos estados de Minas Gerais e São Paulo por ano e mês

# PERGUNTA 2:

### Escreva um comando em SQL que retorne o ticket médio (valor médio de compra) e quantidade de compras realizadas dos usuários que fizeram compras entre 2019 e 2020 e que tiveram mais de 3 compras no período (lembre-se de contar todas as compras dos usuários que estão nessa condição!)

# PERGUNTA 3:

### Escreva um comando em SQL que retorne quais são os usuários por estado que mais compraram em todo o período analisado e, o número de compras realizadas, a quantidade total de itens comprados e valor total pago

# PERGUNTA 4:

### Escreva um comando em SQL que retorne quais são os 3 produtos mais comprados dos estados da região Sul e Sudeste, a quantidade de itens comprados, o valor total pago e a média de preço paga

# PERGUNTA 5:

### Escreva um comando em SQL que retorne o ticket médio e o número total de usuários que fizeram pelo menos uma compra por faixa etária

# PERGUNTA 6:

### Escreva um comando em SQL que aplique um desconto de 10% em todas as vendas dos usuários que fizeram pelo menos 3 compras de produtos na mesma categoria, a partir da 4ª compra realizada. Exiba apenas os usuários que terão o desconto aplicado, mantendo todas as compras, o valor original e o valor com o desconto aplicado

# PERGUNTA 7

### Escreva um comando em SQL que retorna o produto, quantos usuários fizeram pelo menos uma compra desse produto e qual é o valor mínimo e máximo pago

# PERGUNTA 8

### Escreva um comando em SQL que retorne o total de compras realizadas, o total de itens comprados e a receita total obtida

# PERGUNTA 9

### Escreva um comando em SQL que retorne o produto, a quantidade de itens vendidos e a receita do produto que tem mais itens vendidos por ano e mês

# PERGUNTA 10

### Escreva um comando em SQL que exiba quem são os usuários que compraram produtos diferentes

# PERGUNTA 11

### Escreva um comando em SQL que verifique usuários que não fizeram nenhuma compra